# Advanced Techniques

Master advanced Claude Code techniques to build sophisticated workflows and automation pipelines.

## Prerequisites

Before diving into this notebook, you should be familiar with:
- Claude Code basics (Notebook 01)
- Sub-agents (Notebook 02)
- Hooks (Notebook 03)
- Skills (Notebook 04)
- Slash Commands (Notebook 05)
- MCP Integrations (Notebook 06)
- Tools (Notebook 07)
- Git Workflows (Notebook 08)
- Project Setup (Notebook 09)

This notebook brings everything together with advanced patterns and real-world use cases.

## Advanced Prompt Engineering

### Structuring Complex Requests

**Basic Request (Less Effective):**
```
"Add user authentication to the app"
```

**Advanced Request (More Effective):**
```
"Add user authentication to the app with these requirements:

1. Authentication Method:
   - Use JWT tokens
   - Support email/password and OAuth (Google, GitHub)
   - Include refresh token rotation

2. Security:
   - Hash passwords with bcrypt (cost factor 12)
   - Implement rate limiting (5 attempts per 15 minutes)
   - Add CSRF protection

3. Implementation:
   - Follow existing code patterns in auth/
   - Add middleware for protected routes
   - Create comprehensive tests

4. Documentation:
   - Update API docs with new endpoints
   - Add authentication flow diagram
   - Include setup instructions

Start by analyzing the current auth/ directory structure."
```

### Progressive Refinement Pattern

Instead of asking for everything at once, use progressive refinement:

```
Step 1: "Analyze the current authentication implementation"
Step 2: "Design a JWT-based auth system that integrates with our current setup"
Step 3: "Implement the core JWT authentication"
Step 4: "Add OAuth providers (Google, GitHub)"
Step 5: "Add rate limiting and security measures"
Step 6: "Write comprehensive tests"
Step 7: "Update documentation"
```

### Context-Rich Prompts

Provide context for better results:

```
"I'm seeing a performance issue in our API:

Context:
- Endpoint: GET /api/users
- Response time: ~3 seconds (should be <500ms)
- Database: PostgreSQL 14
- Current users: ~100,000 records
- Framework: Express.js with Sequelize ORM

Symptoms:
- Slow queries in database logs
- High memory usage during requests
- No database indexes on user table

Please:
1. Analyze the current implementation in api/routes/users.js
2. Identify performance bottlenecks
3. Propose optimizations (indexing, pagination, caching)
4. Implement the fixes
5. Add performance tests to prevent regression"
```

## Complex Multi-Step Workflows

### Workflow 1: Complete Feature Development

A typical feature development workflow combining all Claude Code features:

```markdown
## Feature: User Profile Management

### Step 1: Planning (Plan Mode)
[Shift+Tab twice to enter Plan Mode]
"Analyze the current user model and related code. How should we implement
profile editing with avatar upload, profile visibility settings, and activity feed?"

### Step 2: Create Feature Branch
"Create a new branch called 'feature/user-profile-management'"

### Step 3: Database Changes
/generate-migration "Add profile fields to users table"

### Step 4: Backend Implementation
"Implement the profile API endpoints:
- GET /api/profile/:userId
- PUT /api/profile/me
- POST /api/profile/avatar
- PATCH /api/profile/visibility

Include:
- Input validation with Joi
- Authorization checks
- File upload handling for avatars
- Rate limiting"

### Step 5: Frontend Implementation
"Create React components for profile management:
- ProfileView (display profile)
- ProfileEdit (edit form)
- AvatarUpload (image upload with preview)
- PrivacySettings (visibility controls)

Follow existing component patterns in src/components/"

### Step 6: Testing
/generate-tests "Create tests for profile API endpoints and React components"

### Step 7: Run Tests
"Run all tests and fix any failures"

### Step 8: Documentation
/document "Update API documentation for new profile endpoints"

### Step 9: Code Review
/review "Review all changes for security, performance, and best practices"

### Step 10: Create PR
"Create a pull request with a comprehensive description"
```

### Workflow 2: Bug Investigation and Fix

```markdown
## Bug: Memory leak in WebSocket connections

### Investigation Phase
[Plan Mode]
"Search for WebSocket-related code and analyze connection handling:
1. Find all WebSocket connection code
2. Look for event listener registration
3. Check for cleanup in disconnect handlers
4. Identify potential memory leaks"

### Reproduction
"Create a test that reproduces the memory leak by:
1. Opening multiple WebSocket connections
2. Monitoring memory usage
3. Closing connections
4. Verifying memory is released"

### Fix Implementation
"Fix the memory leak by:
1. Adding proper event listener cleanup
2. Implementing connection tracking
3. Adding disconnect handlers
4. Ensuring no circular references"

### Validation
"Run the reproduction test to verify the fix"

### Regression Prevention
"Add automated tests that will catch similar issues in the future"
```

## Combining All Features

### Complete Automation Pipeline Example

Let's build a complete CI/CD-like automation using hooks, skills, commands, and MCP:

#### 1. Hook Configuration (`.claude/settings.json`)

```json
{
  "hooks": {
    "session-start": ".claude/hooks/session-start.sh",
    "before-write": ".claude/hooks/validate-file.sh",
    "after-write": ".claude/hooks/auto-format.sh",
    "pre-commit": ".claude/hooks/pre-commit.sh",
    "post-commit": ".claude/hooks/post-commit.sh"
  },
  "mcp": {
    "servers": {
      "github": {
        "command": "mcp-server-github",
        "env": {
          "GITHUB_TOKEN": "${GITHUB_TOKEN}"
        }
      },
      "slack": {
        "command": "mcp-server-slack",
        "env": {
          "SLACK_TOKEN": "${SLACK_TOKEN}"
        }
      },
      "database": {
        "command": "mcp-server-postgres",
        "env": {
          "DATABASE_URL": "${DATABASE_URL}"
        }
      }
    }
  }
}
```

#### 2. Session Start Hook (`.claude/hooks/session-start.sh`)

```bash
#!/bin/bash
set -e

echo "🚀 Starting development session..."

# Pull latest changes
git fetch origin
BEHIND=$(git rev-list HEAD..origin/main --count)
if [ $BEHIND -gt 0 ]; then
  echo "⚠️  Your branch is $BEHIND commits behind main"
fi

# Check dependencies
if [ -f package.json ]; then
  echo "📦 Checking npm dependencies..."
  npm outdated || true
fi

# Start services
echo "🐳 Starting Docker services..."
docker-compose up -d

# Run health checks
echo "🏥 Running health checks..."
./scripts/health-check.sh

echo "✅ Development environment ready!"
```

#### 3. Auto-Format Hook (`.claude/hooks/auto-format.sh`)

```bash
#!/bin/bash

FILE=$FILE_PATH

case $FILE in
  *.py)
    echo "🐍 Formatting Python: $FILE"
    black $FILE
    isort $FILE
    ;;
  *.js|*.jsx|*.ts|*.tsx)
    echo "📜 Formatting JavaScript/TypeScript: $FILE"
    prettier --write $FILE
    eslint --fix $FILE || true
    ;;
  *.json)
    echo "📋 Formatting JSON: $FILE"
    jq '.' $FILE > $FILE.tmp && mv $FILE.tmp $FILE
    ;;
esac
```

#### 4. Pre-Commit Hook (`.claude/hooks/pre-commit.sh`)

```bash
#!/bin/bash
set -e

echo "🔍 Running pre-commit checks..."

# Linting
echo "  📝 Linting..."
npm run lint

# Type checking
echo "  🔤 Type checking..."
npm run type-check

# Unit tests
echo "  🧪 Running unit tests..."
npm run test:unit

# Security scan
echo "  🔒 Security scan..."
npm audit --audit-level=moderate

# Check for secrets
echo "  🔐 Checking for secrets..."
gitleaks detect --staged || exit 1

echo "✅ All pre-commit checks passed!"
```

## Performance Optimization Techniques

### Optimizing File Operations

**Bad: Multiple individual reads**
```
"Read file1.js, file2.js, file3.js, file4.js, and file5.js"
```

**Good: Use Glob for discovery, then targeted reads**
```
"Find all JavaScript files in src/components/ and read the ones related to authentication"
```

### Optimizing Search Operations

**Bad: Broad search**
```
"Search for 'user' in the entire codebase"
```

**Good: Targeted search**
```
"Search for 'user' in *.js files in the api/ directory, showing matches in context"
```

### Parallel Operations

When operations don't depend on each other, ask Claude to run them in parallel:

```
"In parallel:
1. Run the test suite
2. Run the linter
3. Build the documentation
4. Generate the coverage report

Report results from all four operations."
```

### Caching Strategies

For repeated operations, provide context to avoid re-analysis:

```
"You already analyzed the authentication system in our previous conversation.
Based on that analysis, now add 2FA support to the existing JWT implementation."
```

## Token Management Strategies

### Understanding Token Usage

Tokens are consumed by:
- Your prompts
- Claude's responses
- File contents (when read)
- Tool outputs
- Conversation history

### Strategy 1: Conversation Segmentation

For large projects, segment work into focused conversations:

```
Session 1: Database schema design
[Complete and /clear]

Session 2: Backend API implementation
[Complete and /clear]

Session 3: Frontend implementation
[Complete and /clear]

Session 4: Testing and documentation
[Complete and /clear]
```

### Strategy 2: Use /compact Wisely

Use `/compact` when:
- You're 70%+ through context window
- Switching to a related task
- The conversation has tangents you don't need

Don't use `/compact` when:
- You'll need specific details from history
- You're in the middle of complex implementation

### Strategy 3: Reference Files by Path

Instead of pasting code, reference files:

**Bad (wastes tokens):**
```
"Here's my code:
[pastes 500 lines]
Can you optimize it?"
```

**Good (efficient):**
```
"Analyze and optimize the code in src/services/DataProcessor.js"
```

### Strategy 4: Targeted File Reading

Use line ranges for large files:

```
"Read lines 100-150 of src/utils/helpers.js where the calculateTax function is defined"
```

## Advanced Debugging Techniques

### Systematic Debugging Workflow

```markdown
## Bug: API returns 500 error on user registration

### Step 1: Gather Information
"Show me:
1. The user registration endpoint code
2. Recent error logs
3. The user model/schema
4. Database migration files for users table"

### Step 2: Reproduce
"Create a test that reproduces this error"

### Step 3: Isolate
"Add detailed logging to each step in the registration flow:
- Request validation
- Password hashing
- Database insertion
- Email sending
- Response generation"

### Step 4: Identify Root Cause
"Run the test with detailed logging and identify where it fails"

### Step 5: Fix
"Fix the identified issue and ensure the test passes"

### Step 6: Prevent Regression
"Add tests that would catch this issue in the future"
```

### Using Sub-Agents for Debugging

Create a debugging sub-agent:

```markdown
---
name: debugger
description: Systematic debugging and root cause analysis
allowed-tools: [Read, Grep, Bash(pytest:*), Bash(npm test:*)]
model: sonnet
---

You are an expert debugger who solves issues systematically.

For each bug:

1. **Reproduce**: Create a minimal test case
2. **Isolate**: Add logging to narrow down the issue
3. **Analyze**: Examine related code and data
4. **Hypothesis**: Form theories about the root cause
5. **Test**: Validate each hypothesis
6. **Fix**: Implement the solution
7. **Verify**: Ensure the fix works
8. **Prevent**: Add tests to prevent regression

Always provide:
- Root cause explanation
- Why it wasn't caught earlier
- How to prevent similar issues
```

## Real-World Advanced Use Cases

### Use Case 1: Automated Refactoring

```markdown
## Task: Refactor codebase from CommonJS to ES Modules

### Planning
"Analyze the codebase and create a refactoring plan:
1. Find all files using CommonJS (require/module.exports)
2. Identify dependencies between files
3. Determine migration order (dependencies first)
4. Flag any potential issues"

### Implementation
"Execute the refactoring plan:
1. Update package.json with \"type\": \"module\"
2. Migrate files in dependency order
3. Update import statements
4. Fix any dynamic imports
5. Update test files
6. Update build configuration"

### Validation
"Validate the refactoring:
1. Run all tests
2. Check for import errors
3. Verify build succeeds
4. Test in development environment"
```

### Use Case 2: Database Migration

```markdown
## Task: Migrate from MongoDB to PostgreSQL

### Phase 1: Schema Design
"Analyze MongoDB collections and design PostgreSQL schema:
1. Review all Mongoose models
2. Design normalized PostgreSQL tables
3. Create migration scripts
4. Plan for schema evolution"

### Phase 2: Data Migration
"Create data migration scripts:
1. Export data from MongoDB
2. Transform data for PostgreSQL
3. Import with validation
4. Verify data integrity"

### Phase 3: Code Migration
"Update application code:
1. Replace Mongoose with Sequelize/Prisma
2. Update all queries
3. Handle transactions properly
4. Update connection pooling"

### Phase 4: Testing
"Comprehensive testing:
1. Unit tests for all models
2. Integration tests for queries
3. Performance testing
4. Load testing"
```

### Use Case 3: Microservices Extraction

```markdown
## Task: Extract payment processing into microservice

### Phase 1: Analysis
"Analyze payment processing code:
1. Identify all payment-related code
2. Map dependencies
3. Design service API
4. Plan database changes"

### Phase 2: Create Microservice
"Build payment microservice:
1. Set up new service project
2. Implement API endpoints
3. Set up database
4. Implement authentication
5. Add monitoring/logging"

### Phase 3: Integration
"Integrate with main application:
1. Update main app to call payment service
2. Handle service communication
3. Implement circuit breakers
4. Add retry logic"

### Phase 4: Migration
"Migrate existing payments:
1. Sync historical data
2. Dual-write during transition
3. Verify consistency
4. Cut over to microservice"
```

## Practical Exercises

### Exercise 1: Build Complete Automation (90 min)

**Objective**: Create a complete automation pipeline for your project

**Tasks**:
1. Set up hooks for all lifecycle events
2. Create custom skills for common tasks
3. Build slash commands for workflows
4. Configure MCP integrations (GitHub, Slack)
5. Test the complete pipeline

**Validation**:
- All hooks execute correctly
- Skills can be invoked properly
- Commands work end-to-end
- MCP integrations connect successfully

### Exercise 2: Advanced Refactoring (60 min)

**Objective**: Perform a complex refactoring across the codebase

**Tasks**:
1. Identify a pattern to refactor (e.g., callback hell to async/await)
2. Find all occurrences
3. Create a refactoring plan
4. Implement changes systematically
5. Validate with tests

**Validation**:
- All instances are refactored
- Tests still pass
- No regressions introduced

### Exercise 3: Performance Optimization (60 min)

**Objective**: Optimize a slow part of your application

**Tasks**:
1. Identify performance bottleneck
2. Profile the code
3. Implement optimizations
4. Benchmark improvements
5. Add performance tests

**Validation**:
- Measurable performance improvement
- No functional regressions
- Performance tests prevent future regressions

## Tips and Tricks

### Tip 1: Use CLAUDE.md for Project Context

Create a comprehensive CLAUDE.md in your project root:

```markdown
# Project Context for Claude Code

## Project Overview
E-commerce platform built with Node.js, React, and PostgreSQL.

## Architecture
- **Backend**: Express.js REST API
- **Frontend**: React with Redux
- **Database**: PostgreSQL 14
- **Cache**: Redis
- **Queue**: RabbitMQ

## Key Patterns
- Controllers in `api/controllers/`
- Services in `api/services/`
- Models in `api/models/`
- React components in `client/src/components/`

## Coding Standards
- Use async/await (no callbacks)
- Joi for validation
- Jest for testing
- ESLint for linting

## Common Commands
- `npm run dev` - Start development
- `npm test` - Run tests
- `npm run migrate` - Run migrations
```

### Tip 2: Create Project Templates

Maintain templates for common tasks:

```
.claude/templates/
  ├── api-endpoint.js
  ├── react-component.jsx
  ├── test-suite.test.js
  ├── database-migration.js
  └── documentation.md
```

Reference them in requests:
```
"Create a new API endpoint following the template in .claude/templates/api-endpoint.js"
```

### Tip 3: Maintain a Todo List

For multi-session work, use TodoWrite to track progress:

```
"Create a todo list for implementing user authentication:
1. Design database schema
2. Create migration
3. Implement JWT token generation
4. Create login/register endpoints
5. Add authentication middleware
6. Create frontend login form
7. Add tests
8. Update documentation"
```

### Tip 4: Version Your Configuration

Keep `.claude/` directory in git:

```bash
# .gitignore
# DON'T ignore .claude/
# DO ignore local overrides
.claude/settings.local.json
```

### Tip 5: Create Debug Commands

Build commands for common debugging tasks:

```markdown
---
name: debug-api
description: Debug API issues
---

Debug the API issue:

1. Check recent error logs
2. Review relevant endpoint code
3. Check database connection
4. Verify environment variables
5. Test with curl
6. Add debug logging if needed
7. Reproduce the issue
8. Identify root cause
```

## Key Takeaways

### 1. Comprehensive Integration
- Combine hooks, skills, commands, and MCP for complete automation
- Each feature has its place in the workflow
- Integration multiplies effectiveness

### 2. Progressive Complexity
- Start simple, add complexity as needed
- Break large tasks into smaller steps
- Build reusable components

### 3. Performance Matters
- Optimize file operations
- Use targeted searches
- Manage tokens wisely
- Parallel operations when possible

### 4. Debugging is Systematic
- Follow a process
- Gather information first
- Isolate the issue
- Fix and prevent regression

### 5. Automation Saves Time
- Invest time in setup
- Reap benefits long-term
- Share with team
- Continuously improve

### 6. Context is King
- CLAUDE.md provides project context
- Templates ensure consistency
- Documentation helps everyone

### 7. Security First
- Never commit secrets
- Use environment variables
- Review before committing
- Scan for vulnerabilities

### 8. Test Everything
- Write tests first
- Automate testing
- Prevent regressions
- Aim for high coverage

### 9. Iterate and Improve
- Start with basics
- Add features as needed
- Learn from experience
- Optimize continuously

### 10. Share Knowledge
- Document your workflows
- Share skills and commands
- Help team members
- Build a knowledge base

## Next Steps

### Continue Learning
- Explore Claude Code documentation: https://docs.claude.com/claude-code
- Join the community: https://github.com/anthropics/claude-code
- Read the changelog for new features

### Practice Projects
1. **Automation Challenge**: Fully automate your development workflow
2. **Integration Project**: Connect Claude Code with all your tools (GitHub, Slack, Database, etc.)
3. **Custom Extensions**: Build custom MCP servers for your specific needs
4. **Team Setup**: Configure Claude Code for your entire team

### Advanced Topics
- Multi-repository workflows
- Enterprise deployment patterns
- Custom AI agents for specific domains
- Performance monitoring and optimization
- Security hardening

### Contributing
- Share your skills and commands
- Contribute to MCP server ecosystem
- Write blog posts about your workflows
- Help others in the community

## Congratulations!

You've completed the Claude Code learning series! You now have the knowledge to:
- Use all Claude Code features effectively
- Build sophisticated automation workflows
- Optimize your development process
- Create custom integrations and extensions
- Work efficiently on large-scale projects

**Keep experimenting, keep learning, and happy coding!** 🚀

## Quick Reference

### Advanced Patterns Cheat Sheet

```markdown
## Complete Feature Development
1. Plan in Plan Mode
2. Create feature branch
3. Implement with hooks auto-formatting
4. Run tests with pre-commit hooks
5. Create PR with GitHub MCP
6. Notify team via Slack MCP

## Performance Optimization
- Use Glob for file discovery
- Targeted Grep searches
- Parallel tool execution
- Reference files by path
- Use /compact wisely

## Debugging Workflow
1. Gather information
2. Reproduce issue
3. Isolate with logging
4. Identify root cause
5. Fix and verify
6. Add regression tests

## Token Management
- Segment conversations
- Use /compact at 70%
- Reference files by path
- Use line ranges for large files

## Automation Pipeline
- session-start: Setup environment
- before-write: Validate
- after-write: Auto-format
- pre-commit: Run checks
- post-commit: Notify team
```